## Ortho Prediction Software Developing by krish:

#### main / Global Variable use in this program

In [ ]:
#Blastp = /usr/bin/blastp
Blastp = "blastp"
Blastp_data =  './blastp_data/'
cpu_count = 1 # Default value
#genome = None
genome = ""
infinite_loop = 70
inflation_factor = 1.4
#mode = None
mode = None
Cluster_out = "./cluster_out/"
#threshold_score : 0
threshold_score =0
Save_raw_blastp_score = False
Score_file = "./score_file"
Species = "./species/"
verbose = False

## 1. Read_Species_List()

In [6]:
import glob
species = "./species/"
def Read_Species_List(pr=1):
    global species
    """ If pr is 1, it will print "Species_List". """ 
    read_species =glob.glob(species+"*")   
    selected_species_dic = {}
    backward_selected_species_dic = {}
    #number = 0
    for i, species in enumerate(sorted(read_species), start=1):
        selected_species_dic[i] = species.split('/')[-1]
        backward_selected_species_dic[species.split('/')[-1]] = i 
        #if pr == 1 :
            #print (str(i)+".", species.split('/')[-1])
        number = i
    return selected_species_dic, backward_selected_species_dic, number


In [12]:
def read_species(pr =1):
    import os
    read_species = os.listdir("species/")
    for i,species in enumerate(read_species):
        print(i+1,j)
read_species()

1 AAE
2 CAC
3 ECO
4 ECU
5 HIN
6 LLA
7 SCE
8 SPO
9 SPY
10 SYN
11 TMA
12 YPE


In [19]:
import glob
glob.glob("species/""*")

['species\\AAE',
 'species\\CAC',
 'species\\ECO',
 'species\\ECU',
 'species\\HIN',
 'species\\LLA',
 'species\\SCE',
 'species\\SPO',
 'species\\SPY',
 'species\\SYN',
 'species\\TMA',
 'species\\YPE']

## 1. GetMatrixNumber()

In [ ]:
#The Following matrix files must be in Folder
import os
def GetMatrixNumber():
    "This Function will Return the Matrix name choosed by User"
    import sys
    print ("\n1. BLOSUM45\n2. BLOSUM62\n3. BLOSUM82\n4. Quit")

    metrix_num = input("\nEnter a matrix number: ")
    if metrix_num not in ("1","2",'3'):
        print("Wrong input *%s*Sorry not in list\n"%metrix_num,"*"*20,"Good Bye","*"*20,"\n")
        sys.exit(1)
    if metrix_num =="1":
        return "BLOSUM45"
    if metrix_num == "2":
        return "BLOSUM62"
    if metrix_num == "3":
        return "BLOSUM82"
#GetMatrixNumber()

### 2. GetQuerySequence(genome)

In [ ]:
species = "./species/"
def GetQuerySequence(genome):
    """This Function Read Fastaq Files and return as a list Format with gene Position as a index
    Later I will reduce code Lengths and Speed"""
    gene_sequence=""
    gene_sequence_list= []
    try:
        with open(species+genome) as gene:
            for each_line in gene:
                if ">" in each_line:
                    if gene_sequence != "":                        
                        gene_sequence_list.append(gene_sequence)
                        gene_sequence = ""
                gene_sequence = gene_sequence+each_line
            gene_sequence_list.append(gene_sequence)                   
        return gene_sequence_list                        
                                            
    except IOError as err:
        print ("IOError occurred in GetQuerySequence function : " + str(err)    )
b = GetQuerySequence("AAE")

In [ ]:
b[0]

## 3.Write Query(query, parallel_num)

In [ ]:
def WriteQuery(query, parallel_num):
    
    try:
        with open("./query"+"_"+str(parallel_num)+".csv", "w") as write_query:
            write_query.write(query)
    except IOError as err:
        print ("IOError occurred in WriteQuery function : " + str(err))

In [ ]:
WriteQuery("krish is from ",10)

## 4. RunBlast( Subject , parallel_num)

In [ ]:
import subprocess
blastp_matrix = "BLOSUM62"
def RunBlast(subject, parallel_num):
    "By this Function it will create a Pipe line to run Blastp in Computer by input Parameter"
    subject = Species+subject       
    run_blastp =subprocess.Popen([Blastp, "-query", str(parallel_num), "-subject", subject,
                                "-matrix", blastp_matrix, "-outfmt", "10 qseqid sseqid score length"],
                                 stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    run_blastp_stream = run_blastp.communicate()
    run_blastp_output_stream = run_blastp_stream[0]
    run_blastp_error_stream = run_blastp_stream[1]  
    
    display(run_blastp)
    display(run_blastp_error_stream)
    return run_blastp_output_stream
#RunBlast("AAE","AAE")

## Get_Same_Species_Forward_Best_Hit(blastp_score):

In [ ]:
def Get_Same_Species_Forward_Best_Hit(blastp_score):
    blastp_score_split_list  = []
    temp_best_score = ["-1",'-1','-1']
    second_temp_best_score = []
    best_score = []
    second_best_score = []
    blastp_score_split = blastp_score.split('\n')
    del blastp_score_split[-1] #delete of [''] in last index 